### TO-DO

fix the model issue -> get nice models trained on rolling windows to represent time changes<br>
save all models - > load all models <br>
<br>
interpret each model for each bin: model1 - bin 1 to 9; model2 - bin 1 to 9, etc.<br>
PDP, ALE, correlation matrix<br>
LIME<br>

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras

import pickle
import matplotlib.pyplot as plt

# import shap

In [3]:
dtype_specification = {'SICCD': str}

In [4]:
df = pd.read_csv('less_messy_data_keyword_LESS.csv', dtype=dtype_specification)

In [5]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Date', 'permno', 'return', 'mom1m',
       'mom12m', 'chmom', 'indmom', 'mom36m', 'turn', 'mvel1', 'dolvol', 'ill',
       'zerotrade', 'baspread', 'retvol', 'idiovol', 'beta', 'betasq', 'ep',
       'sp', 'agr', 'nincr', 'return(t-1)', 'r(t+1)', 'SICCD', 'COMNAM'],
      dtype='object')

In [6]:
df = df.drop(columns = ['COMNAM', 'Unnamed: 0.1', 'Unnamed: 0', 'r(t+1)' , 'return'])

In [7]:
df = df.dropna()

In [8]:
df.set_index('Date', inplace=True, drop = True)

In [9]:
df = df.drop(columns = ['SICCD', 'permno'])

In [10]:
df

,mom1m,mom12m,chmom,indmom,mom36m,turn,mvel1,dolvol,ill,zerotrade,baspread,retvol,idiovol,beta,betasq,ep,sp,agr,nincr,return(t-1)
Date,,,,,,,,,,,,,,,,,,,,
2001-02-28,-1.574474,0.371761,0.494096,-0.336094,-0.671895,-0.346706,-0.249674,0.184594,-0.110229,-0.187966,-0.065948,0.090430,0.094682,-1.147764,-0.824350,0.109832,-0.005714,1.352752,3.271570,-1.577213
2001-02-28,-0.955058,0.649673,-0.761935,0.007627,-0.390796,-0.534303,2.462294,1.587350,-0.110587,-0.187966,-0.286938,-0.301630,-0.367442,-1.584168,-0.903136,0.164941,-0.435813,1.039952,0.044234,-0.928233
2001-02-28,-0.093663,2.181353,1.062388,2.002047,-0.847184,-0.456468,3.941007,1.606840,-0.110588,-0.187966,0.135358,0.305177,-0.149938,-1.742606,-0.911117,0.435438,-0.135575,0.262103,0.851068,-0.033864
2001-02-28,0.146188,0.572134,-0.220966,2.002047,-0.384159,-0.448831,0.362108,1.108485,-0.110572,-0.187966,-0.516517,-0.529877,-0.525400,-1.545497,-0.899520,0.194226,-0.139399,0.122298,0.044234,0.129661
2001-02-28,-0.147475,1.592410,0.196068,-0.512919,-0.819860,-0.461310,-0.269049,-0.330978,-0.107913,-0.187966,0.408779,1.033574,0.428061,-0.104307,-0.298094,0.300888,2.164993,0.340622,0.044234,-0.006530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31,0.405079,-0.474713,0.194469,-0.098352,-0.236953,-0.237796,-0.262832,0.083649,-0.110260,-0.187966,-0.805841,-1.015327,-0.431124,0.177004,-0.074674,0.230596,0.006506,0.368983,-0.762600,0.347492
2019-12-31,0.548052,-1.416202,-0.108469,-0.802591,-0.591572,1.138001,-0.178698,0.855936,-0.110546,-0.187966,0.317743,0.838015,-0.164419,1.588338,1.568930,0.191346,1.109432,0.476213,-0.762600,0.581698
2019-12-31,0.079381,-0.722938,0.613486,-0.339810,-0.276255,-0.820610,-0.277718,-1.194736,-0.090019,-0.187965,-0.313432,0.022378,-0.641827,-0.058042,-0.263729,0.146997,-0.516974,0.089618,0.044234,0.098223


### read pickle

In [11]:
bins = pd.read_pickle('dataframes.pkl')

In [12]:
# len of each bin
for item in bins:
    print(len(item))
    print('------')

227
------
2270
------
11350
------
23154
------
6129
------
9988
------
17706
------
6356
------
5675
------


In [13]:
print(bins[0].columns)
print(len(bins[0].columns))

Index(['mom1m', 'mom12m', 'chmom', 'indmom', 'mom36m', 'turn', 'mvel1',
       'dolvol', 'ill', 'zerotrade', 'baspread', 'retvol', 'idiovol', 'beta',
       'betasq', 'ep', 'sp', 'agr', 'nincr', 'return(t-1)'],
      dtype='object')
20


### load models

In [14]:
@keras.saving.register_keras_serializable()
def r2_score_wo_demeaning_nn(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - 0))
    r2 = 1 - (ss_res / ss_tot)
    return r2

In [15]:
with open('models_NN.pkl', 'rb') as f:
    models_NN = pickle.load(f)

In [16]:
# NN

from keras import layers
from keras import Sequential

In [17]:
# for i in range(len(bins)):
#     bins[i] = bins[i].values

### interpreting
***
ALE<br>
correlation matrix<br>

In [18]:
features = ['mom1m', 'mom12m', 'chmom', 'indmom', 'mom36m', 'turn', 'mvel1',
       'dolvol', 'ill', 'zerotrade', 'baspread', 'retvol', 'idiovol', 'beta',
       'betasq', 'ep', 'sp', 'agr', 'nincr', 'return(t-1)']

In [109]:
giant_list_of_ales = list()
giant_list_of_mappingx = list()

**for model 1**

In [19]:
model = models_NN[0]

In [30]:
n_features = 3
feature_idxs = [7, 10, 16]
sample_size = 10


# Calculate the ALE values
ale_of_all_bins_features_intervals = list()
itv_for_all_bins_and_feature = list()

for df in bins:  # for each bin
    
    ale_of_all_features = list()
    itv_for_each_feature = list()

    for i in range(n_features):  # for each feature
        x_idx = [7, 10, 16]
        feature_of_interest = feature_idxs[i]
        print("feature: ", feature_of_interest)

        # create intervals
        feature_values = bins[0].iloc[:, feature_of_interest]
        quantiles = np.quantile(feature_values, [0, 0.2, 0.4, 0.6, 0.8, 1.0])
        minv = min(feature_values)
        maxv = max(feature_values)
        itv = quantiles.tolist()
        itv_for_each_feature.append(itv)
        print("created intervals for feature: ", feature_of_interest)

        samplex = list()  # list initiate: sample drawn from intervals of the current feature of interest
        ale_of_all_intervals = list()   # a list of interval ales

        for i in range(len(itv) - 1):  # for each interval get some sample data
            itv_start = itv[i]
            itv_end = itv[i + 1]

            mask = (df.iloc[:,7] >= itv_start) & (df.iloc[:,7] < itv_end)
            
            if mask.sum() >= sample_size:
                sample = df[mask].sample(sample_size)
            else:
                sample = df[mask]
                
#             sample = df[mask].sample(sample_size)
            xarray = sample.values  # convert to narray
            samplex.append(sample)  # the whole sample list
            to_be_averaged = list()
            print("sampled data for interval: ", i+1)


            for i in range(len(sample)):  # for each sample in that sample interval
                x = xarray[i]   # convert to narray
                print("selected instance: ", x)   # print out the instance of interest

                # permute data
                x_lower = np.array([itv_start if i == feature_of_interest else x[i] for i in range(20)])
                x_upper = np.array([itv_end if i == feature_of_interest else x[i] for i in range(20)])     

                diff = model.predict(x_upper.reshape(1, -1))[0][0] - model.predict(x_lower.reshape(1, -1))[0][0]
                to_be_averaged.append(diff)

            ale_of_this_interval = np.mean(to_be_averaged)
            print("ale of interval: ", i+1, " is ", ale_of_this_interval)

            ale_of_all_intervals.append(ale_of_this_interval)

        ale_of_all_features.append(ale_of_all_intervals)

    ale_of_all_bins_features_intervals.append(ale_of_all_features)
    itv_for_all_bins_and_feature.append(itv_for_each_feature)

feature:  7
created intervals for feature:  7
sampled data for interval:  1
selected instance:  [-0.25686387  0.79360037 -0.19202364  1.45883765 -0.75303107 -0.84790492
 -0.28196197 -1.61997902 -0.0495409   0.66362671 -1.19717577 -0.90010852
  0.26646183 -1.14299582 -0.82302913 -1.07139556  2.24195103  0.07259441
  0.85106818 -0.25774982]
1/1 [==============================] - 0s 30ms/step
selected instance:  [ 2.44296273  0.22502812  1.37528982  0.4881386  -1.00887677 -0.28193934
 -0.2739048  -0.71546104 -0.10952336 -0.1879656   0.38212808  0.45426789
  0.26532149 -0.12864216 -0.31579337  0.04347023  0.56649873 -0.21161655
 -0.76259956  2.03288497]
1/1 [==============================] - 0s 42ms/step
selected instance:  [ 0.10120265 -0.70109945  0.49333065 -1.19394853 -0.4498102  -0.83680144
 -0.28259746 -1.60093288  0.17834693  1.68553736 -0.12292461  1.37387284
  0.35787396 -1.73224171 -0.91093097  0.36166673  1.81347002  0.29971093
  0.04423431  0.25251545]
1/1 [====================

1/1 [==============================] - 0s 46ms/step
selected instance:  [-1.47143684 -0.58730839 -1.22263388 -2.20560954  2.29344699  0.36036618
 -0.26369547  0.14600407 -0.11027595 -0.1879656   0.20054313 -0.18428217
  0.38371436  0.08672754 -0.15014564  0.60009409 -0.09970712 -0.82909518
 -0.76259956 -1.48368751]
1/1 [==============================] - 0s 47ms/step
selected instance:  [-0.32885401  0.51853638  0.14161756  0.01310268  0.41709072 -0.15303847
 -0.25430223  0.15308627 -0.11015273 -0.18796559 -0.23339196 -0.35773304
  0.4407599  -0.18281684 -0.35426571  0.42078617  0.04594504 -0.16888254
  0.04423431 -0.30661588]
1/1 [==============================] - 0s 40ms/step
selected instance:  [-0.83403658 -0.27733689 -0.13069926 -0.12315565 -0.10517422 -0.15794075
 -0.25800074  0.10675396 -0.11035892 -0.18796559 -0.75524172 -0.70863269
  0.16216025 -0.3497831  -0.46475829  0.29110876 -0.12307725  0.07927245
 -0.76259956 -0.82316164]
1/1 [==============================] - 0s 52ms/st

1/1 [==============================] - 0s 38ms/step
selected instance:  [ 0.3403601   1.6751215  -1.08188654 -0.0707948   0.81996035  4.46080083
 -0.25555272  0.86885257 -0.11048536 -0.18796561  0.43545568  0.25346064
  0.17245511  0.34525061  0.07549944  0.36614627  0.16464228 -0.12782828
  0.04423431  0.35441765]
1/1 [==============================] - 0s 36ms/step
selected instance:  [ 0.05683203 -0.62503108  0.05713427 -1.05578871 -0.26885799  0.21513765
 -0.20742737  0.70549287 -0.1104902  -0.1879656  -0.55482074 -0.39744602
 -0.59080252 -0.76247878 -0.68552197  0.20160244 -0.33743384 -0.01563964
  0.04423431  0.05589808]
1/1 [==============================] - 0s 39ms/step
selected instance:  [ 0.35450991  0.52176513 -0.77799593 -0.7134747   0.668807    1.45958474
 -0.18344475  0.90012181 -0.11053903 -0.1879656   0.05444482 -0.18575362
 -0.23526802 -0.64735637 -0.63143524  0.21413388 -0.36538244 -0.11073664
  0.04423431  0.33284203]
1/1 [==============================] - 0s 35ms/st

1/1 [==============================] - 0s 39ms/step
ale of interval:  6  is  -0.002206434
sampled data for interval:  3
selected instance:  [-0.5094643  -1.23457576 -0.92855643 -0.87070982  5.41482837  0.43948136
 -0.27830711 -0.28395538 -0.10847909 -0.1879656   0.51481859  0.37965767
  1.02628702 -0.56576484 -0.58959003  0.56578987  0.08993024 -0.5371169
 -0.76259956 -0.42844743]
1/1 [==============================] - 0s 37ms/step
selected instance:  [-0.62064138 -1.29948927 -0.72039784 -0.67194099  6.8432942   0.3308776
 -0.27871181 -0.35902689 -0.1088939  -0.1879656   0.54133186  0.23955761
  1.03183624 -0.5447136  -0.57832086  0.55644383  0.08993024 -0.5371169
 -0.76259956 -0.5465513 ]
1/1 [==============================] - 0s 45ms/step
selected instance:  [ 0.26607664 -1.20790782 -0.89588819 -0.31403306  4.9865631   0.15080667
 -0.27845427 -0.28392419 -0.10844932 -0.18796559  0.10885625 -0.24615472
  1.06106314 -0.29981138 -0.43296795  0.54549558  0.08993024 -0.5371169
 -0.7625995

1/1 [==============================] - 0s 35ms/step
selected instance:  [-0.52915832 -0.49116994  0.31284588  0.19691055  0.89943625  0.20885856
 -0.20552198  0.71076053 -0.11052329 -0.1879656  -0.37301584 -0.62873656
 -0.14182344 -0.64416173 -0.62985162  0.44498894 -0.24128512 -0.27587063
 -0.76259956 -0.51337923]
1/1 [==============================] - 0s 34ms/step
selected instance:  [-0.08833341 -0.25836733 -0.21533521 -1.6150559  -0.52343432 -0.06469844
 -0.20443694  0.54642607 -0.11051665 -0.1879656  -0.43411821 -0.52104531
 -0.57068493 -0.78187661 -0.69406449  0.20160244 -0.33743384 -0.01563964
 -0.76259956 -0.08528989]
1/1 [==============================] - 0s 35ms/step
selected instance:  [ 1.34321139 -0.88004416  0.75605167 -1.98046963  2.35436172  1.46193905
 -0.25709072  0.39967726 -0.11041425 -0.18796561  0.09091318  0.26058292
  0.49364146 -0.06208985 -0.26677297  0.60009409 -0.09970712 -0.82909518
  0.04423431  1.20171225]
1/1 [==============================] - 0s 37ms/st

...

1/1 [==============================] - 0s 38ms/step
selected instance:  [ 1.02973868 -0.16438124  0.33920468  1.72550278 -0.49804276  0.1749478
 -0.15702687  0.8611543  -0.11054514 -0.1879656  -0.74450983 -0.71834283
 -0.23378041 -1.85612689 -0.91008084  0.23310741 -0.10548251  0.03082409
  0.85106818  0.88279918]
1/1 [==============================] - 0s 35ms/step
selected instance:  [-0.95402149 -0.50637526  1.40484534 -1.16658106 -0.06383453  1.67222015
 -0.18846155  1.07784002 -0.11054102 -0.1879656  -0.10479778 -0.25289851
 -0.72932969 -0.39726853 -0.49395433  0.33319038  0.48615238  0.22257508
 -0.76259956 -0.92345383]
1/1 [==============================] - 0s 35ms/step
selected instance:  [ 1.25456531 -0.72183467 -0.43560415 -0.81385685 -0.73242433  1.14705678
 -0.18941538  0.80993616 -0.11056143 -0.18796561 -0.5619592  -0.54297195
 -0.28140272 -0.61794035 -0.6166847   0.19165032 -0.37795017  0.2743393
 -0.76259956  1.07272992]
1/1 [==============================] - 0s 35ms/step

1/1 [==============================] - 0s 44ms/step
selected instance:  [-1.63671912 -1.50406259  0.78754558  0.23825131 -1.31071931  6.35783522
 -0.2823761  -0.5184322  -0.10592241 -0.1879656   0.89364424  0.66919263
  2.52563673  0.97423273  0.74663436  0.14783834  3.32886089  0.35938648
 -0.76259956 -1.58879524]
1/1 [==============================] - 0s 45ms/step
ale of interval:  10  is  0.050204776
sampled data for interval:  3
selected instance:  [ 0.40726044  0.06924662  1.22046339 -0.00399676 -0.89949048 -0.03445718
 -0.27222298 -0.33018751 -0.1035695  -0.18796555  0.16988546 -0.42080125
 -0.30160008 -0.98862308 -0.77488228 -0.46198619 -0.39347491  0.69776891
  0.04423431  0.3690238 ]
1/1 [==============================] - 0s 35ms/step
selected instance:  [-0.82777428  0.20679769 -1.03050994  0.55288496 -0.19258045 -0.05792213
 -0.27743435 -0.40922245 -0.10889316 -0.1879656   0.01411068 -0.43680117
  0.41312879  0.10395791 -0.13601627  0.06747653  0.32463314  0.57663775
 -0.762

1/1 [==============================] - 0s 35ms/step
selected instance:  [-4.96714874  0.85666123 -2.23817013 -1.43510691 -0.93895635  1.30010419
 -0.27051481  0.50352972 -0.11003842 -0.18796561  4.92041827  4.2195166
  3.00880543 -1.98460993 -0.88834525 -0.18576036 -0.58825067  0.26793351
 -0.76259956 -6.05550732]
1/1 [==============================] - 0s 46ms/step
selected instance:  [ 0.80266398  1.30021543 -3.26681479  0.26546552  1.23204147  4.14701679
 -0.27230324  0.68042567 -0.11012652 -0.18796561  1.27116142  0.36088015
  2.91072987  1.85097424  1.97098866  0.15517203 -0.12845772 -0.28830303
  0.04423431  0.75920532]
1/1 [==============================] - 0s 42ms/step
selected instance:  [-0.10179752 -0.01174695  0.38804714  0.23951141 -0.11103288 -0.20271758
  0.13224067  0.98378807 -0.11057853 -0.1879656  -0.79679333 -0.35056141
 -0.38193614 -0.85007586 -0.72279125  0.21267949 -0.34257468 -0.577214
  0.85106818 -0.0914326 ]
1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 38ms/step
selected instance:  [ 0.46416014  0.38843979  0.54331376  0.44778757 -0.42490437 -0.27889255
 -0.26194006 -0.1180273  -0.11033493 -0.1879656  -0.53349242 -0.76210015
  0.63248914  0.05930953 -0.17236129  0.12858172  1.8209995   0.27651469
  0.85106818  0.40249321]
1/1 [==============================] - 0s 43ms/step
selected instance:  [-2.16816481  0.55980651 -1.36444172 -1.70125957 -0.13674028  0.59422346
 -0.27794305 -0.15419272 -0.10896777 -0.18796561  3.11559667  4.97221935
  0.52575798  0.33555384  0.06650793  0.21015878  0.46354108 -1.97595299
  0.04423431 -1.01777012]
1/1 [==============================] - 0s 42ms/step
selected instance:  [-0.83394282 -1.83385189 -0.77786887 -1.10520126 -0.93083141  2.70630078
 -0.28102118 -0.18803477 -0.10740455 -0.1879656   0.94265129  0.54980429
  1.02739407  1.00019255  0.77805417 -0.81369853 -0.48191431  1.0154474
 -0.76259956 -0.7278953 ]
1/1 [==============================] - 0s 37ms/ste

1/1 [==============================] - 0s 38ms/step
selected instance:  [-0.40226614  0.23892396 -2.08494071 -0.1047634   3.77999891  4.31825494
 -0.2727781   0.24681078 -0.10965072 -0.18796561  2.08011812  1.72660449
  2.90345381  1.82610172  1.93161848  0.15517203 -0.12845772 -0.28830303
 -0.76259956 -0.14562186]
1/1 [==============================] - 0s 43ms/step
selected instance:  [ 0.93622085 -0.2451776  -0.31370651 -0.66672706 -0.28824169  0.63880393
  0.09717224  1.18041    -0.11057391 -0.1879656  -0.3210294   0.26971147
 -0.85954609 -1.02456938 -0.78702493  0.26623116 -0.27482314  0.35110585
 -0.76259956  0.86713419]
1/1 [==============================] - 0s 39ms/step
selected instance:  [-0.25755005 -0.25712046 -0.02082985 -0.12638107 -0.04697311  0.07916653
 -0.13740741  0.7534204  -0.11057024 -0.1879656  -0.87951505 -0.95561454
 -0.76260259 -1.09880311 -0.81031141  0.19926226 -0.17794263  0.07951951
 -0.76259956 -0.26009231]
1/1 [==============================] - 0s 38ms/st

In [38]:
ale_of_all_bins_features_intervals[0] # ALE values for bin 1

[[-0.02966107, -0.016014528, 0.016507275, 0.00628526, -0.010376021],
 [-0.0069779595, -0.002206434, 0.0009347058, 0.029779142, 0.010376021],
 [0.003203166, 0.0, 0.0, 0.009307105, nan]]

In [40]:
itv_for_all_bins_and_feature[0][0][:-1]   # interval for bin 1

[-2.074324200217632,
 -0.6182728067146941,
 0.07411313307413152,
 0.36349348112416197,
 0.6012606904091867]

**generate mapping x**

In [43]:
dfs = itv_for_all_bins_and_feature
mappingx = list()

for i in range(9):
    # for each df
    df = itv_for_all_bins_and_feature[i]
    xaxis = list()
    
    for i in range(3):
        interval_bounds = df[i]

        # Calculate midpoints
        midpoints = [(interval_bounds[i] + interval_bounds[i+1]) / 2 for i in range(len(interval_bounds)-1)]
        # print(midpoints)
        midpoints_list = midpoints
        xaxis.append(midpoints_list)
    
    mappingx.append(xaxis)

In [45]:
import pickle

In [46]:
pd.to_pickle(ale_of_all_bins_features_intervals, 'ALEvalue_model1.pkl')

In [47]:
pd.to_pickle(mappingx, 'ALEx_model1.pkl')

In [143]:
ale_of_all_bins_features_intervals[0]

[[0.02046069, -0.009627954, -0.007229006],
 [0.0, 0.005721074, nan],
 [0.0, 0.03458674, nan]]

In [144]:
mappingx[0]

[[-1.5363979169136548, -0.4605453503056999, 0.6153072163022552],
 [-0.5846901828952511, 0.6402809965704988, 1.8652521760362486],
 [0.11103098062902969, 1.0638578264976535, 2.016684672366277]]

In [145]:
ale_of_all_bins_features_intervals[8]

[[0.12183402, -0.75290537, -0.013529011],
 [-0.07183672, 0.0, -0.23104155],
 [-0.19964425, 0.040248897, 0.10376021]]

In [146]:
mappingx[8]

[[-1.5363979169136548, -0.4605453503056999, 0.6153072163022552],
 [-0.5846901828952511, 0.6402809965704988, 1.8652521760362486],
 [0.11103098062902969, 1.0638578264976535, 2.016684672366277]]